In [ ]:
include("instance_generation_no_x.jl")
using LinearAlgebra

# Task 1: implementing the large-scale model

## Model construction 

In the following, the full model must be implemented and solved using Cbc.

In [ ]:
TotalFacilities = 5
TotalClients = 10
TotalScenarios = 20

instance = generate_instance(TotalFacilities, TotalClients, TotalScenarios)
fullmodel = generate_full_problem(instance)
optimize!(fullmodel)

In [ ]:
# Examine the solutions
@show y_bar = value.(fullmodel[:y])
@show obj = objective_value(fullmodel)

In [ ]:
## Benders decomposition

## Generates the main problem
function generate_and_solve_lagrangian_subproblem(instance, λ)

    I, J, S, K, P, O, V, U, T, D, bigM = unroll_instance(instance)
    S⁻ = 2:length(S)

    lag_sub = Model(HiGHS.Optimizer)
    set_silent(lag_sub)

    # Decision variables
    @variable(lag_sub, 0 <= y[I, S] <= bigM) # Capacity decided for facility i ∈ I
    @variable(lag_sub, w[I, J, S] >= 0) # Flow between facility i ∈ I and client j ∈ J in scenario s ∈ S
    @variable(lag_sub, z[J, S] >= 0) # Shortage in location j ∈ J in scenario s ∈ S


    # Capacity limits: cannot deliver more than capacity decided, 
    #   and only if facility was located
    @constraint(lag_sub, capBal[i in I, s in S],
        sum(w[i, j, s] for j in J) <= y[i, s]
    )

    # Demand balance: Demand of active clients must be fulfilled
    @constraint(lag_sub, demBal[j in J, s in S],
        sum(w[i, j, s] for i in I) >= D[j, s] - z[j, s]
    )

    @objective(lag_sub, Min,
        sum(P[s] *
            (sum(V[i] * y[i, s] for i in I) +
             sum(T[i, j] * w[i, j, s] for i in I, j in J) +
             sum(U * z[j, s] for j in J))
            for s in S) +
        sum(
            sum(λ[i, s] * (y[i, 1] - y[i, s]) for i in I)
            for s in S⁻)
    )

    optimize!(lag_sub)

    return value.(y), objective_value(lag_sub)
end

In [ ]:
function update_lagrangian_multipliers(λ, y_s, UB, LB, k, ϵ, no_improvement, α)

    if no_improvement == 10
        println("Decreasing α...")
        α = 0.8 * α
        no_improvement = 0
    end

    TotalServers, TotalScenarios = size(y_s)
    I = 1:TotalServers
    S = 1:TotalScenarios
    S⁻ = 2:TotalScenarios

    for i in I
        for s in S⁻
            β = sum((y_s[i, 1] - y_s[i, s])^2 for s in S⁻) > ϵ ? α * (UB - LB) / sum((y_s[i, 1] - y_s[i, s])^2 for s in S⁻) : 0.0
            λ[i, s] = λ[i, s] + β * (y_s[i, 1] - y_s[i, s])
        end
    end

    return λ, no_improvement, α

end

In [ ]:
#%
function lagrangian_decomposition(ins; max_iter=200)
    k = 1
    ϵ = 1e-4
    λ = zeros(length(ins.I), length(ins.S))
    y_s = zeros(length(ins.I), length(ins.S))
    LB_k = 0.0
    LB = -Inf
    UB = 1100
    no_improvement = 0
    α = 0.05

    start = time()

    while k <= max_iter

        LB_prev = copy(LB_k)
        y_s, LB_k = generate_and_solve_lagrangian_subproblem(ins, λ)

        if LB_k <= LB_prev
            no_improvement += 1
        end

        LB = max(LB_k, LB)

        λ_prev = copy(λ)
        λ, no_improvement, α = update_lagrangian_multipliers(λ, y_s, UB, LB, k, ϵ, no_improvement, α)
        println("Iter $(k): LB: $(round(LB, digits=2))")

        # if norm(λ_prev - λ) < ϵ 
        #     break
        # end

        k += 1
    end
    return y_s, LB
end


In [ ]:
y_s, LB = lagrangian_decomposition(instance, max_iter=1000)
